### Load preprocessed data

In [1]:
import os.path
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile

Run the script that downloads and processes the MovieLens data

In [2]:
#!python ../src/download.py

In [3]:
import numpy as np
fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(train_x)
print(' ')
print(train_y)

[[   1 1193   11   10]
 [   1  914   26   10]
 [   1 3408    7   10]
 ...
 [6040  562   37    6]
 [6040 1096  109    6]
 [6040 1097   99    6]]
 
[[5.]
 [3.]
 [4.]
 ...
 [5.]
 [4.]
 [4.]]


### Define the MF Model

This is the same `AbstractModel` class as from the Intro Pytorch notebook. We'll continue resuing this class.

In [4]:
%%writefile abstract_model.py
import torch
from random import shuffle
import pytorch_lightning as pl


def chunks(X, Y, size):
    """Yield successive n-sized chunks from l."""
    starts = list(range(0, len(X), size))
    # always visit your data in a random order!
    shuffle(starts)
    for i in starts:
        arrs = (X[i:i + size], Y[i:i + size])
        # convert numpy arrays to torch arrays
        arrs = [torch.from_numpy(arr) for arr in arrs]
        yield arrs


class AbstractModel(pl.LightningModule):
    def save_data(self, train_x, train_y, test_x, test_y):
        self.train_x = train_x
        self.train_y = train_y
        self.test_x = test_x
        self.test_y = test_y

    def training_step(self, batch, batch_nb):
        input, target = batch
        prediction = self.forward(input)
        loss = self.likelihood(prediction, target) + self.prior()
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        input, target = batch
        prediction = self.forward(input)
        # Note that we do *not* include the regularization / prior loss
        # at test time
        loss = self.likelihood(prediction, target)
        tensorboard_logs = {'test_loss': loss}
        return {'test_loss': loss, 'log': tensorboard_logs}
    
    def test_epoch_end(self, outputs):
        test_loss_mean = torch.stack([x['test_loss'] for x in outputs]).mean()
        return {'test_loss': test_loss_mean}
    
    def train_dataloader(self):
        return chunks(self.train_x, self.train_y, self.batch_size)

    def test_dataloader(self):
        return chunks(self.test_x, self.test_y, self.batch_size)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-2)

Overwriting abstract_model.py


In [5]:
from abstract_model import AbstractModel

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


class MF(AbstractModel):
    def __init__(self, n_user, n_item, train_x, train_y, test_x, test_y, 
                 k=18, c_vector=1.0, batch_size=128):
        super().__init__()
        self.save_data(train_x, train_y, test_x, test_y)
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.batch_size = batch_size
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        return ui_interaction
    
    def likelihood(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse
    
    def prior(self):
        # Compute L2 reularization over user (P) and item (Q) matrices
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        total = prior_user + prior_item
        return total
    


model = MF(n_user, n_item, train_x, train_y, test_x, test_y)

# add a logger
logger = TensorBoardLogger("tb_logs", name="bottleneck_model")

# We could have turned on multiple GPUs here, for example
# trainer = pl.Trainer(gpus=8, precision=16)    
trainer = pl.Trainer(max_epochs=5,
                     reload_dataloaders_every_epoch=True,
                     logger=logger)    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


### Train model

In [7]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(31.9442)}
--------------------------------------------------------------------------------



{'test_loss': 31.944181442260742}

#### Run model

In [ ]:
trainer.fit(model)
trainer.test(model)


  | Name | Type      | Params
-----------------------------------
0 | user | Embedding | 108 K 
1 | item | Embedding | 71 K  


In [ ]:
model.user.weight.data.numpy()